# Try running clustering using just the first recorder name

See: https://github.com/pensoft/BiCIKL/issues/28

Building the markov chain model for team parsing in this environment took some time (training step)
Loaded a datafile built externally to test the process, overwriting the recordedby value with the extracted forst team member.

In [0]:
do_setup = False
if do_setup:
  recordedby_first_team_member = 'dbfs:/FileStore/tables/agents_first_tm_csv.gz'
  sparkDF = spark.read.csv(recordedby_first_team_member, sep='\t', header="true", inferSchema="true")
  # Save as table
  sparkDF.write.mode('overwrite').saveAsTable('nickyn.occurrence_first_tm')

In [0]:
%sql
USE nickyn;

In [0]:
%sql
DROP TABLE IF EXISTS occurrence;

Add an occurrence table with an extra column holding the first team member name (`first_tm`).

In [0]:
%sql
    CREATE TABLE occurrence 
    AS
    SELECT *
    FROM gbif.occurrence;


num_affected_rows,num_inserted_rows


Run original clustering, storing results in hash, candidates and relationships

In [0]:
%sql
DROP TABLE IF EXISTS hash;
DROP TABLE IF EXISTS candidates;
DROP TABLE IF EXISTS relationships;

In [0]:
%scala
import org.gbif.pipelines.clustering.Cluster

val args = Array("--hive-db", "nickyn", "--hive-table-hashed", "hash", "--hive-table-candidates", "candidates", "--hive-table-relationships", "relationships")
Cluster.main(args)

Configuration: Map('hiveDatabase -> nickyn, 'hiveTableHashed -> hash, 'hiveTableCandidates -> candidates, 'hiveTableRelationships -> relationships)
import org.gbif.pipelines.clustering.Cluster
args: Array[String] = Array(--hive-db, nickyn, --hive-table-hashed, hash, --hive-table-candidates, candidates, --hive-table-relationships, relationships)

How many relationships were created?

In [0]:
%sql
SELECT count(DISTINCT id1), count(*) FROM relationships

count(DISTINCT id1),count(1)
19140426,115174986


In [0]:
%sql
SELECT count(DISTINCT id1), count(*) 
FROM relationships
WHERE reasons LIKE '%SAME_RECORDER_NAME%'

count(DISTINCT id1),count(1)
5342604,47744244


In [0]:
%sql
SELECT occ1.gbifid, occ1.recordedby, occ2.gbifid, occ2.recordedby
FROM occurrence occ1 INNER JOIN relationships rel ON occ1.gbifid = rel.id1
  INNER JOIN occurrence occ2 on rel.id2 = occ2.gbifid
WHERE occ1.recordedby != occ2.recordedby
AND rel.REASONS LIKE '%SAME_RECORDER_NAME%'
LIMIT 50

gbifid,recordedby,gbifid,recordedby


## Use the first team member instead of the full recordedby value

How many `recordedby` values exist?

In [0]:
%sql
SELECT COUNT(DISTINCT first_tm)
FROM occurrence_first_tm

count(DISTINCT first_tm)
989123


Update recordedby with first team member values

In [0]:
%sql
MERGE INTO occurrence occ
USING occurrence_first_tm occftm
ON occ.gbifid = occftm.gbifIDsRecordedBy
WHEN MATCHED AND occftm.first_tm IS NOT NULL THEN
  UPDATE SET occ.recordedby = occftm.first_tm

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
40075774,40075774,0,0


Run clustering, storing results in hash_rb, candidates_rb and relationships_rb

In [0]:
%sql
DROP TABLE IF EXISTS hash_rb;
DROP TABLE IF EXISTS candidates_rb;
DROP TABLE IF EXISTS relationships_rb;

In [0]:
%scala
import org.gbif.pipelines.clustering.Cluster

val args = Array("--hive-db", "nickyn", "--hive-table-hashed", "hash_rb", "--hive-table-candidates", "candidates_rb", "--hive-table-relationships", "relationships_rb")
Cluster.main(args)

Configuration: Map('hiveDatabase -> nickyn, 'hiveTableHashed -> hash_rb, 'hiveTableCandidates -> candidates_rb, 'hiveTableRelationships -> relationships_rb)
import org.gbif.pipelines.clustering.Cluster
args: Array[String] = Array(--hive-db, nickyn, --hive-table-hashed, hash_rb, --hive-table-candidates, candidates_rb, --hive-table-relationships, relationships_rb)

In [0]:
%sql
SELECT count(DISTINCT id1), count(*) 
FROM relationships_rb;

count(DISTINCT id1),count(1)
19148555,114924512


In [0]:
%sql
SELECT count(DISTINCT id1), count(*) 
FROM relationships_rb
WHERE reasons LIKE '%SAME_RECORDER_NAME%'

count(DISTINCT id1),count(1)
5772420,47660604


In [0]:
%sql
SELECT occ1.gbifid, occ1.recordedby, occ2.gbifid, occ2.recordedby
FROM gbif.occurrence occ1 INNER JOIN relationships_rb rel ON occ1.gbifid = rel.id1
  INNER JOIN gbif.occurrence occ2 on rel.id2 = occ2.gbifid
WHERE occ1.recordedby != occ2.recordedby
AND rel.REASONS LIKE '%SAME_RECORDER_NAME%'
LIMIT 50

gbifid,recordedby,gbifid,recordedby
202122577,Aistleitner Eyjolf,18835907,"Aistleitner Eyjolf, Aistleitner Ulrich"
202122473,Aistleitner Eyjolf,18835907,"Aistleitner Eyjolf, Aistleitner Ulrich"
1323824652,"Liv Borgen, Torstein Engelskjøn",21431005,Liv Borgen | Torstein Engelskjøn
111302249,B. Cabezudo & L. EspaÃ±a,29590540,B. Cabezudo & L. España
1419000608,"Bueno, J.; Barba, R.; Contreras, A.",46404516,"Bueno, J. & Barba, R. & Contreras, A."
2807369346,John Taylor|Connie Taylor,177032220,John Taylor; Connie Taylor
1851339243,"Molero Mesa,Pérez Raya & Martínez Parras",196092613,"Molero Mesa, Pérez Raya & Martínez Parras"
1261131986,Thomas B. Croat|Dylan P. Hannon,218943009,Thomas B. Croat
2814928567,Marshall C. Johnston|Fernando Chiang-Cabrera|Tom Wendt|James S. Henrickson,218945999,Marshall C. Johnston
1846509580,Elaine Joyal; M. Silva C.,218997056,Elaine Joyal
